# Time Series
## DatetimeIndex

In [ ]:
# Let's start by importing the packages we use in this chapter
# and by setting the plotting backend to Plotly
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

In [ ]:
# This creates a DatetimeIndex based on a start timestamp,
# number of periods and frequency ("D" = daily).
daily_index = pd.date_range("2020-02-28", periods=4, freq="D")
daily_index

In [ ]:
# This creates a DatetimeIndex based on start/end timestamp.
# The frequency is set to "weekly on Sundays" ("W-SUN").
weekly_index = pd.date_range("2020-01-01", "2020-01-31", freq="W-SUN")
weekly_index

In [ ]:
# Construct a DataFrame based on the weekly_index. This could be
# the visitor count of a museum that only opens on Sundays.
pd.DataFrame(data=[21, 15, 33, 34],
             columns=["visitors"], index=weekly_index)

In [ ]:
msft = pd.read_csv("csv/MSFT.csv")

In [ ]:
msft.info()

In [ ]:
msft.loc[:, "Date"] = pd.to_datetime(msft["Date"])

In [ ]:
msft.dtypes

In [ ]:
msft = pd.read_csv("csv/MSFT.csv",
                   index_col="Date", parse_dates=["Date"])

In [ ]:
msft.info()

In [ ]:
msft.loc[:, "Volume"] = msft["Volume"].astype("float")
msft["Volume"].dtype

In [ ]:
msft = msft.sort_index()

In [ ]:
msft.index.date

In [ ]:
msft.loc["2019", "Adj Close"]

In [ ]:
msft.loc["2019-06":"2020-05", "Adj Close"].plot()

## Working with Time Zones

In [ ]:
# Add the time information to the date
msft_close = msft.loc[:, ["Adj Close"]].copy()
msft_close.index = msft_close.index + pd.DateOffset(hours=16)
msft_close.head(2)

In [ ]:
# Make the timestamps time-zone-aware
msft_close = msft_close.tz_localize("America/New_York")
msft_close.head(2)

In [ ]:
msft_close = msft_close.tz_convert("UTC")
msft_close.loc["2020-01-02", "Adj Close"]  # 21:00 without DST

In [ ]:
msft_close.loc["2020-05-01", "Adj Close"]  # 20:00 with DST

## Shifting and Percentage Changes

In [ ]:
msft_close.head()

In [ ]:
msft_close.shift(1).head()

In [ ]:
returns = np.log(msft_close / msft_close.shift(1))
returns = returns.rename(columns={"Adj Close": "returns"})
returns.head()

In [ ]:
# Plot a histogram with the daily log returns
returns.plot.hist()

In [ ]:
simple_rets = msft_close.pct_change()
simple_rets = simple_rets.rename(columns={"Adj Close": "simple rets"})
simple_rets.head()

## Rebasing and Correlation

In [ ]:
parts = []  # List to collect individual DataFrames
for ticker in ["AAPL", "AMZN", "GOOGL", "MSFT"]:
    # "usecols" allows us to only read in the Date and Adj Close
    # For a refresher about f-strings, see Chapter 3
    adj_close = pd.read_csv(f"csv/{ticker}.csv",
                            index_col="Date", parse_dates=["Date"],
                            usecols=["Date", "Adj Close"])
    # Rename the column into the ticker symbol
    # (If you type this example by hand, make sure to keep the
    # following lines correctly indented!)
    adj_close = adj_close.rename(columns={"Adj Close": ticker})
    # Append the stock's DataFrame to the parts list
    parts.append(adj_close)

In [ ]:
# Combine the 4 DataFrames into a single DataFrame
adj_close = pd.concat(parts, axis=1)
adj_close

In [ ]:
adj_close = adj_close.dropna()
adj_close.info()

In [ ]:
# Use a sample from June 2019 - May 2020
adj_close_sample = adj_close.loc["2019-06":"2020-05", :]
rebased_prices = adj_close_sample / adj_close_sample.iloc[0, :] * 100
rebased_prices.head(2)

In [ ]:
rebased_prices.plot()

In [ ]:
# Correlation of daily log returns
returns = np.log(adj_close / adj_close.shift(1))
returns.corr()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.imshow(returns.corr(),
                x=adj_close.columns,
                y=adj_close.columns,
                color_continuous_scale=list(
                    reversed(px.colors.sequential.RdBu)),
                zmin=-1, zmax=1)
fig.show()

## Resampling

In [ ]:
end_of_month = adj_close.resample("M").last()
end_of_month.head()

In [ ]:
end_of_month.resample("D").asfreq().head()  # No transformation

In [ ]:
end_of_month.resample("W-FRI").ffill().head()  # Forward fill

## Rolling Windows

In [ ]:
# Plot the moving average for MSFT with data from 2019
msft19 = msft.loc["2019", ["Adj Close"]].copy()

# Add the 25 day moving average as a new column to the DataFrame
msft19.loc[:, "25day average"] = msft19["Adj Close"].rolling(25).mean()
msft19.plot()